# Behavioral Object Detection

This notebook details how to use [Detectron2](https://github.com/facebookresearch/detectron2) to build a POC object detection model.

In [ ]:
import os
import random
import json
import boto3
import sagemaker
import s3fs
import cv2
import matplotlib.pyplot as plt

fs = s3fs.S3FileSystem()

In [ ]:
# S3 path to manifest file
BUCKET = 'behavior-images'
FOLDER = 'fps2-output/driver-actions'
MANIFEST = 'manifests/output/output.manifest'

# Define object dictionary
objects = { '0': 'phone',
            '1': 'cigarette',
            '2': 'phub',
            '4': 'smoke'
          }

# Define manifest files
manifest = 's3://{}/{}/{}'.format(BUCKET, FOLDER, MANIFEST)

## Validate and confirm the access to the bucket

In [ ]:
# Make sure the bucket is in the same region as this notebook.
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name
s3 = boto3.client('s3')
bucket_region = s3.head_bucket(Bucket=BUCKET)['ResponseMetadata']['HTTPHeaders']['x-amz-bucket-region']
assert bucket_region == region, "Your S3 bucket {} and this notebook need to be in the same region. (notebook region: {}, bucket region: {})".format(BUCKET, region, bucket_region)

## Process data

We will convert VOC data format to Json format for SageMaker API.

In [ ]:
with fs.open(manifest) as f:
    manifest_list = [json.loads(line.strip()) for line in f.readlines()]

print("{} items are found in the manifest file.".format(len(manifest_list)))

In [ ]:
manifest_list[1]

Download dataset from s3 using s3 high level cli command

In [ ]:
!aws s3 cp s3://behavior-images/fps2-input/ data/fs2-input/ --recursive

Register the dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. 

In [ ]:
from detectron2.structures import BoxMode

def get_data_dicts(manifest_list, img_dir):
    """Prepare datasets for detectron2 with manifest list and image directory

    Args:
        - manifest_list (list): annotation list
        - img_dir (str): local directory where images are stored
    Returns:
        - dataset_dicts: list(dict)
    """
    dataset_dicts = []
    for idx, v in enumerate(manifest_list):
        record = {}
        file_name = v['source-ref'].rsplit('/', 1)[-1]
        img_size = v['driver-actions']['image_size'][0] # idk why img_size is a list of more than one xywh
        annotations = v['driver-actions']['annotations']
        
        record['file_name'] = os.path.join(img_dir, file_name)
        record['image_id'] = idx
        record['height'] = img_size['height']
        record['width'] = img_size['width']
        
        objs = []
        for annot in annotations:
            x = annot['left']
            y = annot['top']
            w = annot['width']
            h = annot['height']
            category_id = annot['class_id']
            obj = {
                'bbox': [x, y, w, h],
                'bbox_mode': BoxMode.XYWH_ABS,
                'category_id': category_id
            }
            objs.append(obj)
        record['annotations'] = objs
        dataset_dicts.append(record)
    return dataset_dicts
        

Register dataset and metadata. Read more in the same (tutorial)[https://detectron2.readthedocs.io/tutorials/datasets.html] about custom dataset. If you already registered it and you want to redo it, use `DatasetCatalog.remove('driver_action')` first.

In [ ]:
DatasetCatalog.remove('driver_action')

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog

DatasetCatalog.register("driver_action", lambda : get_data_dicts(manifest_list, 'data/fs2-input'))
MetadataCatalog.get("driver_actioin").set(thing_classes=['phone', 'cigarette', 'phub', 'smoke'])

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
def cv2_imshow(image):
    # set size
    plt.figure(figsize=(10,10))
    plt.axis("off")

    # convert color from CV2 BGR back to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()

In [ ]:
from detectron2.utils.visualizer import Visualizer

dataset_dicts = get_data_dicts(manifest_list, 'data/fs2-input')
for d in random.sample(dataset_dicts, 3):
    print(d['file_name'])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("driver_actioin"), scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

## Define a model and TRAIN it

In this step, we finetued a COCO-pretrained R50 FPN Faster RCNN model. First we need to specify configurations for it.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")) #Get the basic model configuration from the model zoo 
#Passing the Train and Validation sets
cfg.DATASETS.TRAIN = ("driver_action",)
cfg.DATASETS.TEST = ()
# Number of data loading threads
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# Number of images per batch across all machines.
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0125  # pick a good LearningRate
cfg.SOLVER.MAX_ITER = 300  #No. of iterations   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

Then we simply train it by defining a trainer and call its `train()` method.

In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Check training curves.
Following magic commands have problem runing in SageMaker, follow this [post](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab) for trouble shooting.

Also, you can launch it in a seperate console, and access it with `https://<YOUR_URL>.studio.region.sagemaker.aws/jupyter/default/proxy/6006/`.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output 

## Inference & evaluation

First, let's create a predictor using the model we just trained:

In [ ]:
from detectron2.engine import DefaultPredictor

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
# use the same dataset for training
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("driver_actioin"), 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

### Use AP metrics for evaluation

(this part can't run currently)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("driver_action", ("bbox"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "driver_action")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`